In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [27]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [28]:
X = train_data.drop('label', axis=1, inplace=False)
y = train_data.label

In [29]:
X /= 255.0
test_data /= 255.0

In [30]:
X = X.values.reshape(-1, 28, 28, 1)
test_data = test_data.values.reshape(-1, 28, 28, 1)

In [8]:
ohe = OneHotEncoder()
y = np.array(y)
y = y.reshape(len(y), 1)
ohe.fit(y)
y = ohe.transform(y).toarrayay()

In [9]:
y.shape

(42000, 10)

In [10]:
X_part, X_cv, y_part, y_cv = train_test_split(X, y, test_size=0.15, random_state=44, shuffle=True)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_part, y_part, test_size=0.25, random_state=44, shuffle=True)

In [13]:
image_shape = (28, 28, 1)
DigiModel = keras.models.Sequential([
    keras.layers.Conv2D(filters = 32, kernel_size=(5, 5), activation=tf.nn.relu,
                       padding='same', input_shape=(image_shape)),
    keras.layers.Conv2D(filters = 32, kernel_size=(5, 5), activation=tf.nn.relu,
                       padding='same'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='valid'),
    keras.layers.Dropout(0.25),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=64, kernel_size=(3,3),activation = tf.nn.relu , padding='same'),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3),activation = tf.nn.relu , padding='same'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.BatchNormalization(),
    
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.45),
    keras.layers.Dense(units=10, activation=tf.nn.softmax)
    
])

In [14]:
DigiModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [15]:
DigiModel.fit(X_train, y_train, validation_data=(X_cv, y_cv), epochs=16, batch_size=64, verbose=1)

Epoch 1/16
419/419 [==============================] - 114s 273ms/step - loss: 0.2527 - acc: 0.9200 - val_loss: 0.0729 - val_acc: 0.9800
Epoch 2/16
419/419 [==============================] - 99s 236ms/step - loss: 0.0915 - acc: 0.9738 - val_loss: 0.0773 - val_acc: 0.9778
Epoch 3/16
419/419 [==============================] - 75s 179ms/step - loss: 0.0737 - acc: 0.9780 - val_loss: 0.0746 - val_acc: 0.9813
Epoch 4/16
419/419 [==============================] - 74s 177ms/step - loss: 0.0559 - acc: 0.9830 - val_loss: 0.0477 - val_acc: 0.9870
Epoch 5/16
419/419 [==============================] - 74s 176ms/step - loss: 0.0520 - acc: 0.9843 - val_loss: 0.0519 - val_acc: 0.9854
Epoch 6/16
419/419 [==============================] - 74s 177ms/step - loss: 0.0459 - acc: 0.9860 - val_loss: 0.0462 - val_acc: 0.9878
Epoch 7/16
419/419 [==============================] - 81s 193ms/step - loss: 0.0405 - acc: 0.9884 - val_loss: 0.0452 - val_acc: 0.9894
Epoch 8/16
419/419 [==============================] - 

In [16]:
ModelLoss, ModelAccuracy = DigiModel.evaluate(X_test, y_test)
print(ModelLoss, ModelAccuracy, sep='\n')

279/279 [==============================] - 6s 21ms/step - loss: 0.0646 - acc: 0.9863
0.06461113691329956
0.986330509185791


In [31]:
y_pred = DigiModel.predict(X_test)

In [32]:
FinalResults = DigiModel.predict(test_data)
FinalResults = pd.Series(np.argmax(FinalResults, axis=1), name='Label')
FileSubmission = pd.concat([pd.Series(range(1, 28001), name='ImageId'), FinalResults], axis=1)
FileSubmission.to_csv('submission_3.csv', index=False)